In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [95]:
ticker = 'AFLT3'
df_history = yf.download(tickers=f"{ticker}.SA", period='2y')


[*********************100%***********************]  1 of 1 completed


In [96]:
df_history.index.min()

Timestamp('2019-05-14 00:00:00')